<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Meas Module Reference

This a Jupyter Notebook Reference document for the SLab projects

Version 1.0 (7/3/2019) License information is at the end of the document

---

This **Meas Module Reference Notebook** describes the **Meas Module** commands.  
Those are the commands contained in the **ac.py** source file.


## Module Import

In order to use the **meas** commands, we first need to import the meas module. This is an interactive módule, so, in order to execute the code examples, you should import it executing the cell below.

We will also import the main **SLab** module as it includes some needed commands.

In [ ]:
# Import the main SLab module
import slab

# Import the Meas module
import slab.meas as meas

The examples provided in this document use waves generated using code, not real mesurements. So you don't need to connect to the **Hardware Board** to execute the examples.

# Command Index

The Meas SLab module contains several commands that ease the measurements. The following list contains all the commands in the Meas module. You can use the hiperlinks to jump to one of the commands.

[period](#period) : Computes the period of a signal   
[tcross](#tcross) : Computes when a signal cross a given value   
[analyze](#analyze) : Analyze one or several signals   

<a id='period'></a>

## period

Compute the period of a signal  
Period is computed from signal crossings at the halfrange  

>**period(vector,time,ts,mode)**  
>Required parameters:  
>$\quad$ vector : Sequence of values  
>Optional parameters:    
>$\quad$ time : Optional time vector  
>$\quad$ ts : Optional sample time    
>$\quad$ mode : Cross mode tmodeRise (Default) or tmodeFall      
>Returns a the mean period using:  
>$\quad$ time vector if provided  
>$\quad$ Ts if provided  
>$\quad$ Samples indexes if no time or Ts is provided   

The only required parameter is **vector** the rest of parameters are optional.  
The command obtains the **halfRange** of the vector and determine the locations where the halfRange is crossed in the **mode** direction. Period is computed averaging the differences of the obtained locations.  
Mode can be **slab.tmodeRise** (option by default) or **slab.tmodeFall**.  
If a time array is provided, the command use two times in this array to compute the sample time, so uniform sampling is required to obtain proper results.  
If no time array is provided, and a sampling time **ts** is provided, period is calculated from the provided sample time.
You can use the sample time from the last measurement using the internal variable **slab.sampleTime** if the module has been loaded with the default slab name.  
If no **time** or **ts** parameter is given, period is returned in number of samples.  

If less than two half range crosses are found, the command generates a **"Not enough edges for period"** exception.


In [ ]:
# EXAMPLE : Measure the period of a signal

# Import the numpy module
import numpy as np

# Create a sinewave signal
time  = []
value = []
for i in range(0,500):
    t = i/100.0
    v = 1 + np.sin(2*np.pi*t) 
    time.append(t)
    value.append(v)
    
# Show the signal
slab.plot11(time,value,"1Hz Sinewave",'t(s)','Value')
    
T = meas.period(value,time,mode=slab.tmodeFall)
print('Computed period is',T,'s')

<a id='tcross'></a>

## tcross

Determine the times when a vector crosses a value

>**tcross(vector,value,mode,time,ts)**  
>Required parameters:   
>$\quad$ vector : Sequence of values  
>$\quad$ value : Value to cross  
>Optional parameters:   
>$\quad$ mode : Cross mode tmodeRise (Default) or tmodeFall  
>$\quad$ time : Optional time vector  
>$\quad$ ts : Optional sample time    
>Returns a vector of cross instants:  
>$\quad$ Times if time vector is provided  
>$\quad$ Time from Ts if provided  
>$\quad$ Indexes if no time or Ts is provided   

The required parameters are **vector** and **value**. The rest of parameters are optional.  
The command obtains a list of indexes $i_j$ where the vector values cross value in the direction indicated by mode.  
Direction set by mode can be **slab.tmodeRise** (option by default) or **slab.tmodeFall**.  
If a time array is provided, it will be used to give the time location of the cross.  
If not, and a sampling time ts is provided, location will be given as ts * ij.  
You can use the sample time from the last measurement using the internal variable slab.sampleTime if the module has been loaded with the default slab name.  
If no time or ts parameter is given, the vector indices ij are returned.  

We will explain the operation of the command using the waveform shown below. The command first computes three values from the signal: The **half range**, in green, halfway between wave maximum and minimum values, the **high threshold**, in red, halfway between the half range and the maximum and the **low treshold**, in cyan, halfway between the half range and the minimum.

![signal](images/meas/signal.png)

If the **slab.tmodeRise** mode has been selected, the command explores the wave, from left to right until its value is below the **low treshold**. Then, the cross time is found when the signal crosses the half range. At that point the algorith resets to find the next cross.
If the **slab.tmodeFall** mode has been selected, the algorithm is similar, but this time we check against the signal going over the high treshold.
In the example figure we see that we have found three crosses, indicated with purple, yellow and black vertical lines.


In [ ]:
# EXAMPLE : Obtain the croses of a sine wave

# Import the numpy module
import numpy as np

# Create a sinewave signal
time  = []
value = []
for i in range(0,500):
    t = i/100.0
    v = 1 + np.sin(2*np.pi*t) 
    time.append(t)
    value.append(v)
    
# Show the signal
slab.plot11(time,value,"1Hz Sinewave",'t(s)','Value')
    
vector = meas.tcross(value,1.0,slab.tmodeFall,time)
print('Crosses falling below 1.0 value:',vector)

vector = meas.tcross(value,1.0,slab.tmodeRise,time)
print('Crosses rising above 1.0 value:',vector)

<a id='analyze'></a>

## analyze

Analize signal data and show results on screen

>**analyze(data)**  
>Optional parameters:  
>data : data tuple to analyze  
>$\quad$ Pos 0 : Time vector  
>$\quad$ Pos 1 onward : Signal value vectors   
>If data is not provided, a transientAsync command will be performed   
>Returns nothing  

Performs analysis on one or several signals. The signals are vectors that include a magnitude variation respect to time. The data to be analyzed can be supplied in three ways.  
Data can be a list of floats o a numpy one dimension array. In that case it will represent one signal and time will be measured in samples with one sample for each data element.  
Data can be list of vectors with each vector built as a list of floats or a one dimension numpy array. In that case, the first vector will be considered time and the rest will be considered signals numbered from 1 onwards. Units will be the ones of the vectors.  
If data is not provided, a **transientAsync** command will be issued and its returned data will be used to provide as input data. Unit for time will be seconds and it will be volt for other vectors.

If a time vector is available, the command provides:

* Start time
* End time
* Total time

From each signal in the data, the command provides:

* **Mean** value for all provided points 
* **Standard deviation** value for all provided points
* **High Peak** (maximum of the signal)
* **Low Peak** (minimum of the signal)
* **Peak to peak** value (maximum - minimum)
* **Halfrange** value (halfway between maximum and minimum)
* **RMS** value (without substracting mean value)
* **Mean period** (like the value returned by the period command)

Note that, if the signal is not periodic, the mean period value won't make sense.

Units only will be provided if parameter **data** is not provided and analysis is performed from a **transientAsync** command.


In [ ]:
# EXAMPLE : Analyze two sine waves

# Import the numpy module
import numpy as np

# Create a sinewave signal
time   = []
value1 = []
value2 = []
for i in range(0,500):
    t  = i/100.0
    v1 = 1    +      np.sin(2*np.pi*t) 
    v2 = 0.75 + 0.25*np.sin(np.pi*t) 
    time.append(t)
    value1.append(v1)
    value2.append(v2)
    
# Show the signals
slab.plot1n(time,[value1,value2],"Sine Waves",'t(s)','Values',['V1','V2'])
    
# Analyze the signals
meas.analyze((time,value1,value2))

## Document license

Copyright  ©  Vicente Jiménez (2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">